In [1]:
import os

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

import scipy.interpolate as interpolate
import scipy.constants as constants

In [6]:
%matplotlib widget

In [18]:
base_path = '/Users/manifestation/Stanford/beads/comsol_models/new_trap_efield/near_field/data/'
# base_path = '/USERS/NADAV/PATH/TO/ANALYSIS/FOLDER/'
base_path = r'/home/nadavp/tmp/'
fname = 'new_trap_attractor_shield_potential_efield.txt'


positions_to_sample = np.array([
                                [13.9e-6, 0.0e-6, -15.23e-6], \
                                [30e-6, 0.0e-6, -15.23e-6], \
                                [40e-6, 0.0e-6, -15.23e-6], \
                                [50e-6, 0.0e-6, -15.23e-6], \
                               ])

dipole_units = constants.elementary_charge * 1.0e-6
dipole_moment = np.array([100.0, 0.0, 0.0]) * dipole_units


In [13]:
#############################################################
######  LOAD DATA FROM COMSOL OUTPUT TO NUMPY ARRAYS  #######
#############################################################

path = os.path.join(base_path, fname)

with open(path, 'r') as f:
    lines  =  f.readlines()

for lineind, line in enumerate(lines):
    if 'Grid' in line:
        grid_ind = lineind + 1
    elif 'V (V)' in line:
        voltage_ind = lineind + 1
    elif 'es.Ex (V/m)' in line:
        ex_ind = lineind + 1
    elif 'es.Ey (V/m)' in line:
        ey_ind = lineind + 1
    elif 'es.Ez (V/m)' in line:
        ez_ind = lineind + 1

xvec_str = lines[grid_ind].strip('\n')
yvec_str = lines[grid_ind+1].strip('\n')
zvec_str = lines[grid_ind+2].strip('\n')

xvec = np.array( [float(val) for val in xvec_str.split()] )
yvec = np.array( [float(val) for val in yvec_str.split()] )
zvec = np.array( [float(val) for val in zvec_str.split()] )

lenx = len(xvec)
leny = len(yvec)
lenz = len(zvec)

voltage = np.zeros((lenx, leny, lenz))
efieldx = np.zeros((lenx, leny, lenz))
efieldy = np.zeros((lenx, leny, lenz))
efieldz = np.zeros((lenx, leny, lenz))

for j in range(leny):
    for k in range(lenz):
        voltage_line = lines[voltage_ind + j + k*leny]
        voltage[:,j,k] = np.array( [float(val) for val in voltage_line.strip('\n').split()] ) 

        efieldx_line = lines[ex_ind + j + k*leny]
        efieldx[:,j,k] = np.array( [float(val) for val in efieldx_line.strip('\n').split()] )

        efieldy_line = lines[ey_ind + j + k*leny]
        efieldy[:,j,k] = np.array( [float(val) for val in efieldy_line.strip('\n').split()] )

        efieldz_line = lines[ez_ind + j + k*leny]
        efieldz[:,j,k] = np.array( [float(val) for val in efieldz_line.strip('\n').split()] )





#############################################################
##############  BUILD 3D GRID INTERPOLATORS  ################
#############################################################

voltage_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), voltage)

efieldx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldx)
efieldy_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldy)
efieldz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldz)

In [14]:
#############################################################
############  SIMPLE PLOT AT Z=0 FOR EXAMPLE  ###############
#############################################################

zcoord = 0.0

n_arrow = 9
arrow_xvec = np.linspace(10e-6, 90e-6, n_arrow)
arrow_yvec = np.linspace(-180e-6, 180e-6, n_arrow)
arrow_efieldx = np.zeros((n_arrow,n_arrow))
arrow_efieldy = np.zeros((n_arrow,n_arrow))
for i in range(n_arrow):
    for  j in range(n_arrow):
        arrow_efieldx[i,j] = efieldx_func([arrow_xvec[i], arrow_yvec[j], zcoord])
        arrow_efieldy[i,j] = efieldy_func([arrow_xvec[i], arrow_yvec[j], zcoord])

norm = colors.Normalize(vmin=0, vmax=1)

z_ind = np.argmin(np.abs(zvec - zcoord))

fig, ax = plt.subplots(1,1)

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size='4%', pad=0.1)
cb1 =  mpl.colorbar.ColorbarBase(ax_cb, cmap=mpl.cm.plasma, \
                                 norm=norm, orientation='vertical')
cb1.set_label('Voltage [V]')
fig.add_axes(ax_cb)

ax.set_title('Voltage at $z=0$ plane\nwith attractor biased to 1 V')

ax.contourf(xvec*1e6, yvec*1e6, voltage[:,:,z_ind].T, 100, cmap='plasma', norm=norm)

quiver = ax.quiver(arrow_xvec*1e6, arrow_yvec*1e6, arrow_efieldx[:,:].T, arrow_efieldy[:,:], \
                   color='w', pivot='mid', scale=1e5, label='Efield')
ax.quiverkey(quiver, X=0.7, Y=0.85, U=1.0e4, label='$10^4$ V/m', labelcolor='w')

ax.set_xlabel('X-coordinate [$\\mu$m]')
ax.set_ylabel('Y-coordinate [$\\mu$m]')
fig.tight_layout()
# fig.show()
# plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
gradx_func(point)

array([-1.70323396e+08])

In [20]:
charges = 1e-3
force = charges * 1.6e-19 * 400 / 8e-3 * 0.61  # in Newtons
force

4.8799999999999995e-18

In [21]:
grad_mili = 1e6
grad_mili*dipole_moment[0]

1.602176634e-17

In [127]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

dipole_moment = np.array([100.0, 0.0, 0.0]) * dipole_units

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
for point in positions_to_sample:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()


    MS dipole [e um] :  [100.   0.   0.]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [ 1.22692234e-15 -2.96041324e-16 -2.75802579e-15]



/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

    MS Postion [um]  :  [ 30.     0.   -15.23]
        Efield [V/m] :  [-2267.37095064   -31.10601837  -828.73414718]
Force on dipole [N]  :  [ 1.29930806e-15 -3.14455696e-18 -7.14814877e-16]

    MS Postion [um]  :  [ 40.     0.   -15.23]
        Efield [V/m] :  [-1556.31307815   -42.98086473 -1084.36550179]
Force on dipole [N]  :  [ 9.43004684e-16  3.07722744e-17 -2.22820070e-16]

    MS Postion [um]  :  [ 50.     0.   -15.23]
        Efield [V/m] :  [-1076.59837863   -42.33409451 -1121.51209126]
Force on dipole [N]  :  [ 7.27393329e-16 -6.58035028e-18  1.13438509e-16]



In [ ]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
for point in positions_to_sample:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()

In [117]:
dipole_moment = np.array([0.0, 100.0, 0.0]) * dipole_units
positions_to_sample2 = np.array([[dist_*1e-6, 4.9e-6, -15.23e-6] for dist_ in np.arange(10,50,2.5)])

In [118]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
force_x = []
force_z = []
for point in positions_to_sample2:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()
    force_x.append(force_on_dipole[0])
    force_z.append(force_on_dipole[2])


    MS dipole [e um] :  [  0. 100.   0.]

    MS Postion [um]  :  [ 10.     4.9  -15.23]
        Efield [V/m] :  [-3402.32609917  -153.15188481  1158.27445391]
Force on dipole [N]  :  [-1.04116542e-15 -1.16592266e-15  1.08941043e-15]



/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

    MS Postion [um]  :  [ 12.5    4.9  -15.23]
        Efield [V/m] :  [-3787.98674453  -220.47084801  1232.82975698]
Force on dipole [N]  :  [ 4.27584219e-16  9.64373299e-16 -5.95164667e-16]

    MS Postion [um]  :  [ 15.     4.9  -15.23]
        Efield [V/m] :  [-3543.045309      24.2761565    628.07325318]
Force on dipole [N]  :  [-2.19763890e-16 -4.05362387e-16 -1.28769390e-16]

    MS Postion [um]  :  [ 17.5    4.9  -15.23]
        Efield [V/m] :  [-3380.53314773   -13.96582233   266.31494892]
Force on dipole [N]  :  [-1.82767643e-16 -1.32764694e-16 -1.08284477e-16]

    MS Postion [um]  :  [ 20.     4.9  -15.23]
        Efield [V/m] :  [-3132.94748158   -22.65156466   -52.74740141]
Force on dipole [N]  :  [1.09879050e-16 2.64465229e-16 4.21124232e-17]

    MS Postion [um]  :  [ 22.5    4.9  -15.23]
        Efield [V/m] :  [-2875.23860704   -16.78515443  -341.96730238]
Force on dipole [N]  :  [-7.15031094e-17 -6.07150780e-18  1.62677942e-17]

    MS Postion [um]  :  [ 25.     4.9 

In [120]:
force_x = np.array(force_x)
force_z = np.array(force_z)

In [121]:
_,ax = plt.subplots()
ax.scatter(np.arange(10,50,2.5), force_x/10, label='X-backgrounds')
ax.scatter(np.arange(10,50,2.5), force_z/10, label='Z-backgrounds')
ax.legend()
ax.set(ylabel='Force [N]', xlabel=r'x distance [$\mu m$]', title='MS dipole [e um] : '+str(dipole_moment/dipole_units))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0, 0.5, 'Force [N]'),
 Text(0.5, 0, 'x distance [$\\mu m$]'),
 Text(0.5, 1.0, 'MS dipole [e um] : [  0. 100.   0.]')]

In [137]:
dipole_moment2 = np.array([[0.0, 100.0*np.sin(theta), 100.0*np.cos(theta)] for theta in np.arange(0,2*np.pi,0.2)]) * dipole_units
positions_to_sample3 = np.array([13.9*1e-6, 0.0e-6, -15.23e-6])

In [138]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
force_x2 = []
force_z2 = []
for edm in dipole_moment2:

    sample_efield = np.array([efieldx_func(positions_to_sample3)[0], efieldy_func(positions_to_sample3)[0], efieldz_func(positions_to_sample3)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(positions_to_sample3) * edm[i]
        force_on_dipole[1] += grady_func(positions_to_sample3) * edm[i]
        force_on_dipole[2] += gradz_func(positions_to_sample3) * edm[i]

    print('    MS Postion [um]  : ', positions_to_sample3*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()
    force_x2.append(force_on_dipole[0])
    force_z2.append(force_on_dipole[2])


    MS dipole [e um] :  [100.   0.   0.]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.72888165e-15 -4.42228469e-17 -9.76181112e-16]



/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.75960022e-15  4.42464201e-17 -9.86127840e-16]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.68030223e-15  1.30951722e-16 -9.56760762e-16]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.49414905e-15  2.12436392e-16 -8.89250652e-16]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.20856202e-15  2.85451893e-16 -7.86288925e-16]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-1.83492659e-15  3.47087328e-16 -6.51980340e-16]

    MS Postion [um] 

In [132]:
np.argmax(force_x2), np.argmin(force_x2)

(330, 16)

In [130]:
_,ax = plt.subplots()
ax.scatter(np.arange(0,2*np.pi,0.01), force_x2, label='X-backgrounds')
# ax.scatter(np.arange(0,2*np.pi,0.01), dipole_moment2[:,2], label='X-backgrounds')
# ax.scatter(np.arange(0,2*np.pi,0.01), force_z2, label='Z-backgrounds')
ax.legend()
ax.set(ylabel='Force [N]', xlabel=r'Angle [rad]', title='Spinning in YZ plane')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0, 0.5, 'Force [N]'),
 Text(0.5, 0, 'Angle [rad]'),
 Text(0.5, 1.0, 'Spinning in YZ plane')]

In [139]:
np.sum(force_x2), np.sum(force_z2), np.mean(force_x2), np.mean(force_z2)

(-1.5838231683203737e-15,
 -5.666959708170893e-16,
 -4.949447401001168e-17,
 -1.770924908803404e-17)

In [70]:
len(force_x2)

629

In [170]:
dipole_moment = np.array([0.0, 100.0, 0.0]) * dipole_units
x_att = [50*np.sin(3*t/5000*2*np.pi) for t in range(5000)]
positions_to_sample4 = np.array([[13.9*1e-6, (offset_-4.9)*1e-6, -15.23e-6] for offset_ in x_att])

In [171]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
force_x4 = []
force_z4 = []
for point in positions_to_sample4:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()
    force_x4.append(force_on_dipole[0])
    force_z4.append(force_on_dipole[2])


    MS dipole [e um] :  [  0. 100.   0.]

    MS Postion [um]  :  [ 13.9   -4.9  -15.23]
        Efield [V/m] :  [-3629.04376377   -32.38996578   745.93786973]
Force on dipole [N]  :  [ 1.71677673e-16  1.73685207e-17 -5.89640172e-17]



/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

    MS Postion [um]  :  [ 13.9         -4.71150489 -15.23      ]
        Efield [V/m] :  [-3627.01507648   -32.22025623   745.26695904]
Force on dipole [N]  :  [ 1.66104842e-16  1.87671676e-17 -5.64949343e-17]

    MS Postion [um]  :  [ 13.9         -4.52301245 -15.23      ]
        Efield [V/m] :  [-3624.98641803   -32.0505491    744.59605789]
Force on dipole [N]  :  [ 1.60532090e-16  2.01657947e-17 -5.40258864e-17]

    MS Postion [um]  :  [ 13.9         -4.33452538 -15.23      ]
        Efield [V/m] :  [-3622.95781725   -31.88084679   743.92517581]
Force on dipole [N]  :  [ 1.54959497e-16  2.15643820e-17 -5.15569088e-17]

    MS Postion [um]  :  [ 13.9         -4.14604634 -15.23      ]
        Efield [V/m] :  [-3620.92930296   -31.71115171   743.25432234]
Force on dipole [N]  :  [ 1.49387141e-16  2.29629096e-17 -4.90880364e-17]

    MS Postion [um]  :  [ 13.9         -3.95757801 -15.23      ]
        Efield [V/m] :  [-3619.04582022   -31.4905149    742.63567494]
Force on dipole [N] 

    MS Postion [um]  :  [ 13.9         2.7975821 -15.23     ]
        Efield [V/m] :  [-3774.79189554    70.26226455   702.75288094]
Force on dipole [N]  :  [-2.43751490e-16 -2.16883224e-15  2.04540637e-15]

    MS Postion [um]  :  [ 13.9          2.98377535 -15.23      ]
        Efield [V/m] :  [-3773.48395295    59.43222631   718.24506925]
Force on dipole [N]  :  [-1.03205797e-16 -2.71355366e-15  2.53425059e-15]

    MS Postion [um]  :  [ 13.9          3.16985655 -15.23      ]
        Efield [V/m] :  [-3776.49178359     9.82578936   760.09962681]
Force on dipole [N]  :  [ 2.16611850e-16 -2.36678654e-15  2.27592942e-15]

    MS Postion [um]  :  [ 13.9          3.35582306 -15.23      ]
        Efield [V/m] :  [-3779.90967264   -43.45232314   804.44585172]
Force on dipole [N]  :  [ 5.53353907e-16 -1.93516219e-15  1.94647049e-15]

    MS Postion [um]  :  [ 13.9          3.54167224 -15.23      ]
        Efield [V/m] :  [-3783.3254052    -96.6968204    848.76409687]
Force on dipole [N]  : 

    MS Postion [um]  :  [ 13.9         10.13196084 -15.23      ]
        Efield [V/m] :  [-3586.61051714    -5.83837296   846.13805788]
Force on dipole [N]  :  [-1.45174520e-16  3.98552720e-16  4.39895992e-16]

    MS Postion [um]  :  [ 13.9         10.31162896 -15.23      ]
        Efield [V/m] :  [-3.58770648e+03 -2.82560911e+00  8.42828628e+02]
Force on dipole [N]  :  [-1.65361148e-16  3.15383231e-16  3.31593875e-16]

    MS Postion [um]  :  [ 13.9         10.49108089 -15.23      ]
        Efield [V/m] :  [-3.58880112e+03  1.83529537e-01  8.39523180e+02]
Force on dipole [N]  :  [-1.85523486e-16  2.32313817e-16  2.23422076e-16]

    MS Postion [um]  :  [ 13.9         10.67031408 -15.23      ]
        Efield [V/m] :  [-3.58989443e+03  3.18900022e+00  8.36221761e+02]
Force on dipole [N]  :  [-2.05661248e-16  1.49345661e-16  1.15382132e-16]

    MS Postion [um]  :  [ 13.9         10.84932598 -15.23      ]
        Efield [V/m] :  [-3590.98638282     6.19076024   832.92441891]
Force on di

...

    MS Postion [um]  :  [ 13.9        -35.54535268 -15.23      ]
        Efield [V/m] :  [-3573.47897399   -15.49411133   736.25100448]
Force on dipole [N]  :  [-8.01064262e-17 -2.88807620e-16  6.45563652e-17]

    MS Postion [um]  :  [ 13.9        -35.39619456 -15.23      ]
        Efield [V/m] :  [-3574.82092484   -17.73517044   736.72179402]
Force on dipole [N]  :  [-5.33963746e-17 -3.34713889e-16  8.10582592e-17]

    MS Postion [um]  :  [ 13.9        -35.24660301 -15.23      ]
        Efield [V/m] :  [-3576.16677508   -19.98274153   737.19395157]
Force on dipole [N]  :  [-2.66087100e-17 -3.80753550e-16  9.76081038e-17]

    MS Postion [um]  :  [ 13.9        -35.09658017 -15.23      ]
        Efield [V/m] :  [-3577.51650559   -22.23679268   737.66747041]
Force on dipole [N]  :  [ 2.56187157e-19 -4.26925949e-16  1.14205664e-16]

    MS Postion [um]  :  [ 13.9        -34.94612817 -15.23      ]
        Efield [V/m] :  [-3577.782718     -25.9493695    738.64213978]
Force on dipole [N] 

    MS Postion [um]  :  [ 13.9        -29.26250629 -15.23      ]
        Efield [V/m] :  [-3481.9569799    -41.25068909   820.15034442]
Force on dipole [N]  :  [ 1.84924822e-16 -3.72703375e-16  4.60965322e-16]

    MS Postion [um]  :  [ 13.9        -29.09772746 -15.23      ]
        Efield [V/m] :  [-3478.17428187   -49.6570788    828.32892557]
Force on dipole [N]  :  [ 1.74238836e-16 -3.59956948e-16  4.29452068e-16]

    MS Postion [um]  :  [ 13.9        -28.93260473 -15.23      ]
        Efield [V/m] :  [-3476.06542326   -54.16921722   833.29015073]
Force on dipole [N]  :  [ 1.54428987e-16 -3.21060323e-16  3.84851503e-16]

    MS Postion [um]  :  [ 13.9        -28.76714044 -15.23      ]
        Efield [V/m] :  [-3476.39586891   -53.00660123   833.56181308]
Force on dipole [N]  :  [ 1.21353018e-16 -2.44123999e-16  3.21237554e-16]

    MS Postion [um]  :  [ 13.9        -28.60133695 -15.23      ]
        Efield [V/m] :  [-3476.72699198   -51.84160185   833.83403234]
Force on dipole [N] 

    MS Postion [um]  :  [ 13.9        -22.42671601 -15.23      ]
        Efield [V/m] :  [-3605.3348052    -41.5017093    767.58213777]
Force on dipole [N]  :  [ 6.19945578e-17  6.32413099e-17 -4.01124504e-16]

    MS Postion [um]  :  [ 13.9        -22.25005639 -15.23      ]
        Efield [V/m] :  [-3591.52433246   -35.04574892   757.84757322]
Force on dipole [N]  :  [ 2.76006385e-16  1.67546174e-16 -4.16482732e-16]

    MS Postion [um]  :  [ 13.9        -22.07315018 -15.23      ]
        Efield [V/m] :  [-3577.69458296   -28.58077727   748.0994211 ]
Force on dipole [N]  :  [ 4.90316931e-16  2.71996627e-16 -4.31862397e-16]

    MS Postion [um]  :  [ 13.9        -21.89599991 -15.23      ]
        Efield [V/m] :  [-3572.59037341   -25.61631331   744.11020989]
Force on dipole [N]  :  [ 5.07176807e-16  2.88001597e-16 -3.95044256e-16]

    MS Postion [um]  :  [ 13.9        -21.71860808 -15.23      ]
        Efield [V/m] :  [-3573.63826346   -25.11958638   744.18120097]
Force on dipole [N] 

    MS Postion [um]  :  [ 13.9        -15.19313044 -15.23      ]
        Efield [V/m] :  [-3533.47403696     7.41140681   932.7624945 ]
Force on dipole [N]  :  [-1.03840425e-16  1.15167938e-16  1.83818505e-16]

    MS Postion [um]  :  [ 13.9        -15.00859958 -15.23      ]
        Efield [V/m] :  [-3535.30550972     7.68003142   932.93522845]
Force on dipole [N]  :  [-1.56559539e-16  2.74127486e-17  2.25146509e-17]

    MS Postion [um]  :  [ 13.9        -14.82392505 -15.23      ]
        Efield [V/m] :  [-3537.13840837     7.94886516   933.10809689]
Force on dipole [N]  :  [-1.59016375e-16  2.33231483e-17  1.49975081e-17]

    MS Postion [um]  :  [ 13.9        -14.63910948 -15.23      ]
        Efield [V/m] :  [-3538.97270685     8.21790422   933.28109735]
Force on dipole [N]  :  [-1.59016375e-16  2.33231483e-17  1.49975081e-17]

    MS Postion [um]  :  [ 13.9       -14.4541555 -15.23     ]
        Efield [V/m] :  [-3540.80837909     8.48714477   933.45422738]
Force on dipole [N]  : 

    MS Postion [um]  :  [ 13.9         -7.72592672 -15.23      ]
        Efield [V/m] :  [-3687.62927946   -19.66301023   742.24735843]
Force on dipole [N]  :  [ 4.81589909e-16 -1.96677030e-16  1.34451104e-16]

    MS Postion [um]  :  [ 13.9         -7.53771283 -15.23      ]
        Efield [V/m] :  [-3678.44187435   -23.59549416   745.3523837 ]
Force on dipole [N]  :  [ 4.81036907e-16 -1.86639479e-16  1.25621852e-16]

    MS Postion [um]  :  [ 13.9         -7.34946145 -15.23      ]
        Efield [V/m] :  [-3669.25263933   -27.52876134   748.45802741]
Force on dipole [N]  :  [ 4.80483795e-16 -1.76599929e-16  1.16790842e-16]

    MS Postion [um]  :  [ 13.9         -7.16117525 -15.23      ]
        Efield [V/m] :  [-3660.061705     -31.46275588   751.56424543]
Force on dipole [N]  :  [ 4.79930581e-16 -1.66558523e-16  1.07958199e-16]

    MS Postion [um]  :  [ 13.9         -6.97285692 -15.23      ]
        Efield [V/m] :  [-3651.89457433   -34.79840546   754.1155913 ]
Force on dipole [N] 

In [174]:
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = fft_norm(5000, 5000)

_,ax = plt.subplots(1,2,figsize=(9.2,4))
fft_x = np.abs(np.fft.rfft(force_x4))*scale
fft_z = np.abs(np.fft.rfft(force_z4))*scale
angles_z = np.angle(np.fft.rfft(-force_z4))

freqs_gravity = 3*np.arange(14)
z_to_x = np.array([fft_z[freq1==freq_]/fft_x[freq1==freq_] for freq_ in freqs_gravity])

freq1 = np.fft.rfftfreq(len(force_x4), d=1./5000)
ax[0].semilogy(freq1, fft_x, label='x-backgrounds')
ax[0].semilogy(freq1, fft_z, label='z-backgrounds')
ax[0].set(xlim=(1,70), ylim=(1e-19,1e-14), xlabel='frequency [Hz]', ylabel=r'ASD [N/$\sqrt{Hz.}$]')
ax[0].legend()
ax[1].scatter(freqs_gravity, z_to_x)
ax[1].set(ylabel='Z to X background ratio', xlabel='frequency [Hz.]')
freqs_gravity = 3*np.arange(14)
[print(freq_, angles_z[freq1==freq_]) for freq_ in freqs_gravity]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 [0.]
3 [-1.57079633]
6 [3.14159265]
9 [1.57079633]
12 [-1.01321419e-15]
15 [-1.57079633]
18 [3.14159265]
21 [-1.57079633]
24 [-1.09579889e-15]
27 [-1.57079633]
30 [3.14159265]
33 [-1.57079633]
36 [3.14159265]
39 [1.57079633]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]